In [ ]:
from google.cloud import logging
import datetime
import pytz
import pandas as pd
import re

from logger_utils import log_query, query_logger, generate_log_file, get_scene_log_stats

project_id = "cerulean-338116"
service_name = "cerulean-cloud-test-cr-orchestrator"

# Update to current revision name
revision_name="cerulean-cloud-test-cr-orchestrator-00067-lc8"

# Get uncompleted scenes

In [ ]:
start_time = datetime.datetime.now() - datetime.timedelta(days=1)
query = log_query(service_name, revision_name=revision_name, jsonPayload={"message": "Initiating Orchestrator"}, start_time=start_time)
start_logs = query_logger(project_id, query)
scenes_started = set()
if len(start_logs) > 0:
    start_logs = start_logs.groupby('scene_id').first().reset_index()
    scenes_started = set(start_logs['json_payload'].apply(lambda x: x['scene_id']).unique())

    
query = log_query(service_name, revision_name=revision_name, jsonPayload={"message": "Orchestration complete!"}, start_time=start_time)
finish_logs = query_logger(project_id, query)
scenes_finished = set()
if len(finish_logs) > 0:
    finish_logs = finish_logs.groupby('scene_id').first().reset_index()
    scenes_finished = set(finish_logs['json_payload'].apply(lambda x: x['scene_id']).unique())

if scenes_finished is not None:
    print("Incomplete scenes:")
    display(scenes_started - scenes_finished)

# Get logs for one scene

In [ ]:
logs = get_scene_log_stats(project_id, service_name, revision_name, "S1A_IW_GRDH_1SDV_20230420T175150_20230420T175215_048183_05CB0C_A2E1")
logs['timestamp'] = logs['timestamp'].dt.tz_convert("US/Arizona")
logs.head()

# Generate a log file
This will save a log file from the above logs (log.txt) which shows logs with most recent first

In [ ]:
# saves file as log.txt
generate_log_file(logs, filename="log.txt")

# Get all Errors and Warnings

In [ ]:
# set start time to 1 day ago
start_time = datetime.datetime.now(datetime.UTC) - datetime.timedelta(days=1)

# revision name
revision_name = "cerulean-cloud-test-cr-orchestrator-00026-jzz"

query = log_query(service_name, revision_name=revision_name, start_time=start_time, min_severity="WARNING")
logs = query_logger(project_id, query, df=True)

# filter for only logs created with our manual logger (if the error isn't caught by our logging, this line should be removed)
logs = logs[logs['instanceId'].isnull()==False]

print(len(logs))
display(logs.head())

In [ ]:
generate_log_file(logs, filename="log.txt")

# Get all logs for revision 
(only for testing small number of tiles or this will be very large and probably crash)

In [ ]:
# set start time to 1 day ago
start_time = datetime.datetime.now(datetime.UTC) - datetime.timedelta(days=1)

query = log_query(service_name, revision_name=revision_name, start_time=start_time, min_severity="INFO")
logs = query_logger(project_id, query, df=True)
print(len(logs))

display(logs.head())

# Get all logs for Instance 


In [ ]:
# set start time to 1 day ago
start_time = datetime.datetime.now(datetime.UTC) - datetime.timedelta(days=1)

# revision name
instance_id = "0066d92480de9d8567d5770b9572af423760fcb7c2664ac04080bd831797dffdb0e212ff7d524bdaa59e88e565366cef8de5cb17efbb517960cecdd5532586132bbde998c6ec"

query = log_query(service_name, instance_id=instance_id, start_time=start_time, min_severity="INFO")
logs = query_logger(project_id, query, df=True)
print(len(logs))

display(logs.head())

# Get logs with specific message

In [ ]:
start_time = datetime.datetime.now() - datetime.timedelta(days=1)
query = log_query(service_name, revision_name=revision_name, jsonPayload={"message": "MEMORY USAGE 2"}, start_time=start_time)

mem_logs = query_logger(project_id, query).groupby('scene_id').first().reset_index()

# Old Logs
(This is for the logs prior to structuring and creating jsonPayloads)

In [ ]:
from google.cloud import logging
import datetime
import pytz
import pandas as pd
import re

from logger_utils import log_query, query_logger, generate_log_file

project_id = "cerulean-338116"
service_name = "cerulean-cloud-prod20240903-cr-orchestrator"
revision_name = "cerulean-cloud-prod20240903-cr-orchestrator-00021-zjd"

In [ ]:
start_time = datetime.datetime.now(datetime.UTC) - datetime.timedelta(days=5)
end_time = datetime.datetime.now()

query = log_query(service_name, revision_name=revision_name, start_time=start_time, end_time=end_time, textPayload="Memory limit")
logs = query_logger(project_id, query, df=True)
print(len(logs))

logs.head()

In [ ]:
i = 10
row = logs.iloc[i].to_dict()

start_time = row['timestamp'] - datetime.timedelta(hours=1.5)
end_time = row['timestamp']
instance_id = row['instanceId']

query = log_query(service_name, revision_name=revision_name, instance_id=instance_id, start_time=start_time, end_time=end_time, textPayload="sceneid")
logs1 = query_logger(project_id, query, df=True)
print(len(logs1))

tmp = logs1[logs1['text_payload'].str.contains('sceneid=', na=False)]
tmp['sceneid'] = tmp['text_payload'].apply(lambda x: x.split('sceneid=')[1].split('&')[0].split(' ')[0])
tmp['sceneid'].unique()